#### **Linear Regression**
###### **In this tutorial we will deal with a regression problem and we will learn how to load the data, analyze the data and apply some pre-processing and apply a linear regression model(having only one layer). And further this tutorial we will extend this to a deep neural network. This will help us deep understanding of keras dense layer and activation functions.**

#### **Let's Start**

**Import Necessary Libraries**

In [1]:
# To Silence Warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Import Libraries
import pandas as pd     # used to work with datasets, analyze and modify them
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

# Make numpy printouts easier to read
np.set_printoptions(precision=3, suppress=True)

**Load Dataset**

In [7]:
dataset = pd.read_csv('data.csv')
dataset

,Std_Id,Attendance,Class_Test,Mid,Final,Type
0,LR01,8,8,20,40,Regular
1,LR02,9,7,19,32,Re-Admit
2,LR03,9,9,26,27,Re-Admit
3,LR04,5,8,27,42,Regular
4,LR05,7,5,24,37,Regular
5,LR06,8,5,19,40,Regular
6,LR07,8,5,20,40,Regular
7,LR08,5,9,19,33,Regular
8,LR09,7,10,27,30,Regular
9,LR10,7,8,22,39,Regular


**Clean Data**

In [10]:
# Drop the data with missing values. In our dataset there is no missing value, so we need not drop anything.
#dataset = dataset.dropna()
#dataset

# Convert Categorical 'Type' data into one-hot data: In our dataset, all the data are numerical except Type data. So, to ignore confusion of our model we will one-hot encoding the type.
type = dataset.pop('Type')
dataset['Regular'] = (type == 1)*1
dataset['Re-Admit'] = (type == 2)*1
dataset.head()

KeyError: 'Type'